In [1]:
from Librerias_SAP import SAP_GUI
import win32com.client
import pandas as pd
import subprocess
from time import sleep
from datetime import datetime
from datetime import timedelta

def Clean_Columns(List):
    Set_Columns=[i.strip() for i in List.columns]   #Alert in list
    clean_column=[i if "Unnamed" in i else None for i in List.columns]
    clean_column = list(filter(lambda x: x is not None, clean_column)) 
    List=List.set_axis(Set_Columns, axis=1).drop(clean_column,axis=1)  #Drop Unammed: 0
    return(List)

"""--------------------------------FUNCION PARA LIMPIAR NUMEROS A FLOAT--------------------------------------------------------"""

def Clean_column_number(column):
    column=column.apply(lambda x: float(str(x).strip().replace(",","")))
    return(column)

def default_column(default_columns,dataframe):     #Parametros (default_columns: Columnas predeterminadas,dataframe:)
    diccionay_default_column={}
    if len(default_columns)==len(dataframe.columns):
        for i in range(0,len(default_columns)):
            diccionay_default_column[dataframe.columns[i]]=default_columns[i]   
        dataframe=dataframe.rename(columns=diccionay_default_column)   #Remplazo las columnas con las de default
    else:
        None   #asumo columnas originales como estandar
    return(dataframe)

def Date_Null(dataframe,date):
    filter=(date!="00.00.0000")
    dataframe=dataframe.loc[filter]
    return(dataframe)


"""----------------------------Inciar session----------------------------------------------------"""
# Insert User name and password
Keys=pd.read_excel("C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Claves\\Keys.xlsx")
user=Keys["User"][1]
password=Keys["Password"][1]
# Initialize session
Iniciar=SAP_GUI.SessionSAP(user,password)

# Start transsación
Transsacion="mb52"
provision="GRANELES"   
variant="AUXNVOSPT"
Buscar_MB52_GRANELES=SAP_GUI.Search_MB52(Transsacion,Iniciar,provision,variant)


Name_GRANELES="GRANELES"  # Name file
SAP_GUI.Export_TXT2(Name_GRANELES,Iniciar)

Ruta_GRANELES="C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Archivos_CSV\\"+str(Name_GRANELES)+".txt"
data_GRANELES = pd.read_csv(Ruta_GRANELES, skiprows=1, delimiter='\t')

data_GRANELES=Clean_Columns(data_GRANELES)

default_column_MB52=['Material', 'Texto breve de material', 'Gpo.artíc.', 'Ce.', 'Alm.',
       'Lote', 'UMB', 'Libre utilización', 'En control calidad',
       'Valor libre util.', 'Valor en insp.cal.', 'Bloqueado',
       'Stock no libre', 'Valor stock bloq.', 'Valor no libre', 'Cad./FPC'] #Disposicion PROXI A VENCER

data_GRANELES=default_column(default_column_MB52,data_GRANELES) 


data_GRANELES=Date_Null(data_GRANELES,data_GRANELES['Cad./FPC'])

try:
    data_GRANELES["Alm."]=data_GRANELES["Alm."].astype(int)
    data_GRANELES["Lote"]=data_GRANELES["Lote"].astype(int)
    data_GRANELES['Ce.']=data_GRANELES['Ce.'].astype(int)
except:
    None



"""-------------------------Filtro para eliminar subtotales y combertir fechas-------------------------------"""
data_GRANELES=data_GRANELES[data_GRANELES.Material.notnull()]
data_GRANELES["Cad./FPC"]=data_GRANELES["Cad./FPC"].apply(lambda x: datetime.strptime(x,'%d.%m.%Y'))  #Change columns str to datetime

Data_Denominacion=Clean_Columns(pd.read_csv("C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Documentos 1\\SAP\\SAP GUI\\Denominacion_Articulos.txt",delimiter='\t'))[["Grupo art.","Denominación 2 del gr.artículos"]]
Data_Denominacion=Data_Denominacion.rename(columns={"Denominación 2 del gr.artículos":"Denom.gr.artíc."})
data_GRANELES=pd.merge(Data_Denominacion,data_GRANELES,how="right",left_on="Grupo art.",right_on="Gpo.artíc.")

"""---------------------------------Filtro graneles vencidos-------------------"""
now=datetime.now()
month_vencido=now-timedelta(days=30) 
mask_vencido=(data_GRANELES["Cad./FPC"]<=now) & (data_GRANELES["Cad./FPC"]>=month_vencido)
data_GRANELES_vencidos=data_GRANELES.loc[mask_vencido]
#Sort values
data_GRANELES_vencidos=data_GRANELES_vencidos.sort_values(by=["Cad./FPC"],ascending=False)
data_GRANELES_vencidos=data_GRANELES_vencidos.reset_index(drop=True)
data_GRANELES_vencidos["Cad./FPC"]=data_GRANELES_vencidos["Cad./FPC"].dt.date   #convet datetime to date 



# filter dates between now a 30 month later
now=datetime.now()
month=timedelta(days=30)   # Change limiter day(30 days= 1 month)
month=now+month
mask=(data_GRANELES["Cad./FPC"]>=now) & (data_GRANELES["Cad./FPC"]<=month)
data_GRANELES=data_GRANELES.loc[mask]
#Sort values
data_GRANELES=data_GRANELES.sort_values(by=["Cad./FPC","Libre utilización"])
data_GRANELES=data_GRANELES.reset_index(drop=True)
data_GRANELES["Cad./FPC"]=data_GRANELES["Cad./FPC"].dt.date   #convet datetime to date 


In [2]:
data_Precios_Valores_Junio=pd.read_excel("C:\\Users\\prac.ingindustrial2\\Downloads\\Precios_Valores_Junio.XLSX")

In [3]:
data_GRANELES_informe=pd.merge(data_GRANELES,data_Precios_Valores_Junio,on="Material")

In [4]:
data_GRANELES_informe[['Grupo art.', 'Denom.gr.artíc.', 'Material', 'Texto breve de material',
       'Gpo.artíc.','Valor libre util.', 'Valor en insp.cal.',
       'Bloqueado', 'Stock no libre', 'Valor stock bloq.', 'Valor no libre',
       'Cad./FPC', 'Precio interno periódico', 'Grupo de artículos',
       'Precio estándar', 'Precio interno periódico Estimado']]

,Grupo art.,Denom.gr.artíc.,Material,Texto breve de material,Gpo.artíc.,Valor libre util.,Valor en insp.cal.,Bloqueado,Stock no libre,Valor stock bloq.,Valor no libre,Cad./FPC,Precio interno periódico,Grupo de artículos,Precio estándar,Precio interno periódico Estimado
0,M001,Manufactura,BB-091197A,DTE ROLL-ON EA DEOZ CON FARNES,M001,"659,185",0,0,0,0,0,2023-07-07,2831825,M001,2946686,2831825
1,C014,GR. Prebel,PB0083891,SOMBRA CEJAS CAFE CLARO,C014,"12,645,222",0,0,0,0,0,2023-07-08,1124916,C014,1022901,1124916
2,C014,GR. Prebel,PB0083880,SOMBRA CEJAS CAFE OSCURO,C014,"21,213,180",0,0,0,0,0,2023-07-08,1021558,C014,1030360,1021558
3,M005,Manufactura,Y024220000,RED COLOUR SOLN F/N 8403,M005,"102,177",0,0,0,0,0,2023-07-08,445749,M005,331749,445749
4,C014,GR. Prebel,PB0084313,BASE CREME PUFF,C014,"14,255",0,0,0,0,0,2023-07-09,975958,C014,970403,975958
5,C014,GR. Prebel,PB0068497,POLVO VIT.E TRIGO T3,C014,"2,992,924",0,0,0,0,0,2023-07-09,1421220,C014,1566664,1421220
6,M002,Manufactura,0101535550,MF PESTANINA LASH MAKER CAFE,M002,0,0,22.500,0,"1,998,228",0,2023-07-12,2335580,M002,2137111,2335580
7,M003,Manufactura,PB0026159,BÁLSAMO PARA DESPUÉS DE AFEITARSE,M003,0,"4,746,779",0,0,0,0,2023-07-13,5235306,M003,5003117,5235306
8,C014,GR. Prebel,PB0054057,CREMA SPF 100,C014,0,"316,376,346",0,0,0,0,2023-07-16,14156745,C014,13806582,14156745
9,M002,Manufactura,0101521020,MF HI-FI EYE LINER N°02 MAR RONEXP,M002,0,"1,662,769",0,0,0,0,2023-07-18,1735338,M002,1619135,1735338


In [5]:
list(map(lambda x,y: y if x.strip()!='0' else int(x.strip()),data_GRANELES_informe['Valor no libre'],data_GRANELES_informe["Precio interno periódico Estimado"]))

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [6]:
list(map(lambda x,y: y if x.strip()!='0' else int(x.strip()),data_GRANELES_informe['Valor libre util.'],data_GRANELES_informe["Precio interno periódico Estimado"]))

[2831825,
 1124916,
 1021558,
 445749,
 975958,
 1421220,
 0,
 0,
 0,
 0,
 25181520,
 25181520,
 1497063,
 6372596,
 13179979,
 180739,
 26093499,
 245307,
 11972149,
 3330769,
 43774973,
 1212055,
 18449946,
 32504995,
 790002,
 56315662,
 761291]

In [7]:
"""------------DATAFRAME GRANALES PARA INFORME-----------------------------------------------------------------------"""

data_Graneles_informe=data_GRANELES[['Material', 'Texto breve de material', 'Gpo.artíc.','Denom.gr.artíc.', 'Ce.', 'Alm.',
       'Lote',  'Libre utilización', 'En control calidad','UMB', 'Valor libre util.', 'Valor en insp.cal.','Cad./FPC']]


data_Graneles_Vencidos_informe=data_GRANELES_vencidos[['Grupo art.', 'Denom.gr.artíc.', 'Material', 'Texto breve de material',
       'Gpo.artíc.', 'Ce.', 'Alm.', 'Lote','Bloqueado','Stock no libre', 'UMB','Valor stock bloq.', 'Valor no libre',
       'Cad./FPC']]

data_Graneles_Vencidos_informe

,Grupo art.,Denom.gr.artíc.,Material,Texto breve de material,Gpo.artíc.,Ce.,Alm.,Lote,Bloqueado,Stock no libre,UMB,Valor stock bloq.,Valor no libre,Cad./FPC
0,C038,GR. Avon,1-82524,SANDY BEIGE-ANEW PRESSED POWDE,C038,1000.0,1001.0,10903185,0,0,KG,0,0,2023-07-03
1,C014,GR. Prebel,PB0084331,BASE PAN CAKE,C014,1000.0,1001.0,10900712,6.800,0,KG,"582,262",0,2023-07-02
2,C038,GR. Avon,1-61929,ANEW VITAMIN C SERUM 1-61929800,C038,1000.0,1001.0,10903055,0,224.400,KG,0,"73,295",2023-07-02
3,C038,GR. Avon,1-71814,N3-12 AVON TRUE FLAWLESS ULTRA,C038,1000.0,1001.0,10902264,91,0,KG,"226,360",0,2023-06-30
4,C038,GR. Avon,1-63158,N3-15 NATURAL TAN-AVON TRUE UL,C038,1000.0,1001.0,10902262,95,0,KG,"502,956",0,2023-06-30
5,M005,Manufactura,Y028670000,BLUE COLOUR SOL F/N 6053,M005,1000.0,1001.0,10897571,0,0.430,KG,0,"85,176",2023-06-27
6,C014,GR. Prebel,PB0093601,DEO-AP ROLL-ON ORIGINAL AFM,C014,1000.0,1001.0,10899908,37.800,0,KG,"1,215,133",0,2023-06-24
7,C014,GR. Prebel,PB0074277,POLVO COMP. CALENDULA T4,C014,1000.0,1001.0,10902207,0,13.148,KG,0,"23,071,294",2023-06-24
8,C220,GR.Jerónimo Martens,PB0060364,DEO-AP ROLLON JM FEMENINO FRAG. CLEAN,C220,1000.0,1001.0,10899920,16,0,KG,"3,700,216",0,2023-06-24
9,C266,GR.NOVAVENTA FPT,PB0079251,FRAG. MALE MASS. AMBICIOSO NOVAV,C266,1000.0,1001.0,10890449,0,20,KG,0,"819,418",2023-06-23


In [8]:
"""-----------------------------GENERO SUBTOTALES DE GRANELES------------------------------------------------------------"""

data_GRANELES_Libre=Clean_column_number(data_GRANELES["Valor libre util."]).sum()
data_GRANELES_Calidad=Clean_column_number(data_GRANELES['Valor en insp.cal.']).sum()
Total_Graneles=data_GRANELES_Libre+data_GRANELES_Calidad
Total_Graneles="${:,.2f}".format(Total_Graneles)


"""-----------------------------GENERO SUBTOTALES DE GRANELES VENCIDOS--------------------------------------------------------"""
data_GRANELES_Stock_Bloqueado=Clean_column_number(data_GRANELES_vencidos['Valor stock bloq.']).sum()
data_GRANELES_No_Libre=Clean_column_number(data_GRANELES_vencidos['Valor no libre']).sum()
Total_Graneles_Vencidos=data_GRANELES_Stock_Bloqueado+data_GRANELES_No_Libre
Total_Graneles_Vencidos="${:,.2f}".format(Total_Graneles_Vencidos)




""" Search MB52 raw material"""

# Start transsación
Transsacion="mb52"
provision="MATERIA PRIMA"  
variant="AUXNVOSPT"
Buscar_MB52_MATERIAS=SAP_GUI.Search_MB52(Transsacion,Iniciar,provision,variant)
# Save file



Name_Materias="MATERIAS"  # Revisar suele no guardar
SAP_GUI.Export_TXT2(Name_Materias,Iniciar)
# Pass the route and read file 
Ruta_Materias="C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Archivos_CSV\\"+str(Name_Materias)+".txt"
data_Materias=pd.read_csv(Ruta_Materias,delimiter="\t")
# Change, drop and standardize columns
data_Materias=Clean_Columns(data_Materias)
data_Materias=default_column(default_column_MB52,data_Materias) 
data_Materias=Date_Null(data_Materias,data_Materias['Cad./FPC'])

try:
    # Change Float-Int
    data_Materias["Alm."]=data_Materias["Alm."].astype(int)
    data_Materias["Lote"]=data_Materias["Lote"].astype(int)
    data_Materias['Ce.']=data_Materias['Ce.'].astype(int)
except:
    None



"""-------------------------Filtro para eliminar subtotales y combertir fechas-------------------------------"""
data_Materias=data_Materias[data_Materias.Material.notnull()]
data_Materias["Cad./FPC"]=data_Materias["Cad./FPC"].apply(lambda x: datetime.strptime(str(x),'%d.%m.%Y'))  #Change columns str to datetime

data_Materias=pd.merge(Data_Denominacion,data_Materias,how="right",left_on="Grupo art.",right_on="Gpo.artíc.")


"""---------------------------Buscamos las clases de material------------- """
Data_Material_Total=data_Materias
Lista_tipo_Materias=data_Materias["Gpo.artíc."].drop_duplicates().reset_index(drop=True)


"""--------------------------------------MATERIAS PRIMAS VENCIDAS-------------------------"""

mask_vencido=(data_Materias["Cad./FPC"]<=now) & (data_Materias["Cad./FPC"]>=month_vencido)
data_Materias_vencidos=data_Materias.loc[mask_vencido]
#Sort values
data_Materias_vencidos=data_Materias_vencidos.sort_values(by=["Cad./FPC"],ascending=False)
data_Materias_vencidos=data_Materias_vencidos.reset_index(drop=True)
data_Materias_vencidos["Cad./FPC"]=data_Materias_vencidos["Cad./FPC"].dt.date   #convet datetime to date 


mask=(data_Materias["Cad./FPC"]>=now) & (data_Materias["Cad./FPC"]<=month)
data_Materias=data_Materias.loc[mask]
data_Materias=data_Materias.sort_values(by=["Cad./FPC","Libre utilización"]).reset_index(drop=True)
data_Materias["Cad./FPC"]=data_Materias["Cad./FPC"].dt.date


In [9]:
data_Materias

,Grupo art.,Denom.gr.artíc.,Material,Texto breve de material,Gpo.artíc.,Ce.,Alm.,Lote,UMB,Libre utilización,En control calidad,Valor libre util.,Valor en insp.cal.,Bloqueado,Stock no libre,Valor stock bloq.,Valor no libre,Cad./FPC
0,C272,MP.LOCATEL,PB0083218,FRAG. SENSUAL FLOWER AR680583,C272,1000.0,1001.0,10871504.0,KG,2,0,"208,187",0,0,0,0,0,2023-07-06
1,GRIN1,MP Avon,0-02250000,COCOA BUTTER,GRIN1,1000.0,1001.0,10853194.0,G,"18,888",0,0,0,0,0,0,0,2023-07-06
2,C011,MP. Prebel,Y089210000,MARTINIQUE MANEN 92477 - CPL,C011,1000.0,1001.0,10867992.0,KG,20.144,0,"1,563,445",0,0,0,0,0,2023-07-06
3,C011,MP. Prebel,PB0062306,CAPRYLYL CAPRYLATE/CAPRATE,C011,1000.0,1001.0,10878493.0,KG,2.500,0,"273,930",0,0,0,0,0,2023-07-07
4,C274,MP.BEIERSDORF,PB0084145,LICORICE EXTRACT GW/00199,C274,1000.0,1001.0,10839215.0,KG,4.770,0,"1,188,177",0,1,0,"249,094",0,2023-07-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,C041,MP. Natura,PB0091587,50351573 SANTAL ENJOY 295EA,C041,1000.0,1001.0,10882153.0,KG,10.200,0,"4,727,125",0,0,0,0,0,2023-08-02
69,GRIN1,MP Avon,R172059,DIMETHICONE-VINYL DIM.CROSSPOL,GRIN1,1000.0,1001.0,10891761.0,G,42,0,0,0,0,0,0,0,2023-08-04
70,C272,MP.LOCATEL,PB0080854,BASE HIGH TECH FRESHPLEX MEX-EC0380800,C272,1000.0,1001.0,10871663.0,KG,2.875,0,"344,552",0,0,0,0,0,2023-08-04
71,C011,MP. Prebel,PB0045148,MEX-EC0271361 JABON,C011,1000.0,1001.0,10871662.0,KG,4.099,0,"451,942",0,0,0,0,0,2023-08-04


In [16]:
data_Materias_informe=pd.merge(data_Materias,data_Precios_Valores_Junio,on="Material",how="left")

In [22]:
data_Materias[data_Materias["Grupo art."]=='GRIN1']

,Grupo art.,Denom.gr.artíc.,Material,Texto breve de material,Gpo.artíc.,Ce.,Alm.,Lote,UMB,Libre utilización,En control calidad,Valor libre util.,Valor en insp.cal.,Bloqueado,Stock no libre,Valor stock bloq.,Valor no libre,Cad./FPC
1,GRIN1,MP Avon,0-02250000,COCOA BUTTER,GRIN1,1000.0,1001.0,10853194.0,G,"18,888",0,0,0,0,0,0,0,2023-07-06
10,GRIN1,MP Avon,0-74889,ALUMINA CTD W/CAPRYLSILSESQUIOXANE,GRIN1,1000.0,1001.0,10891475.0,G,"8,688",0,0,0,0,0,0,0,2023-07-10
21,GRIN1,MP Avon,R127197,NATURALS APRICOT & SHEA 913608,GRIN1,1000.0,1001.0,10849675.0,G,"10,000",0,0,0,0,0,0,0,2023-07-13
29,GRIN1,MP Avon,R107840,JAPANESE CHERRY (PRUNUS SERRUL,GRIN1,1000.0,1001.0,10861623.0,G,"8,257",0,0,0,0,0,0,0,2023-07-15
40,GRIN1,MP Avon,0-27430000,TRIOXAUNDECAN ACID 90% A Q,GRIN1,1000.0,1001.0,10894270.0,KG,4.887,0,0,0,0,0,0,0,2023-07-24
41,GRIN1,MP Avon,R153644,POTASSIUM SILICATE-AQ,GRIN1,1000.0,1001.0,10889264.0,G,"52,750",0,0,0,0,0,0,0,2023-07-24
42,GRIN1,MP Avon,0-08560000,SODIUM DTPA,GRIN1,1000.0,1001.0,10872419.0,G,473,0,0,0,0,0,0,0,2023-07-26
43,GRIN1,MP Avon,0-08560000,SODIUM DTPA,GRIN1,1000.0,1001.0,10871321.0,G,600,0,0,0,0,0,0,0,2023-07-26
44,GRIN1,MP Avon,0-03530000,WATERCRESS NASTURTIUM OFFICI-NALE,GRIN1,1000.0,1001.0,10851720.0,KG,3.100,0,0,0,0,0,0,0,2023-07-26
48,GRIN1,MP Avon,0-04600000,MICROCRYSTALLINE PETROLEUM W,GRIN1,1000.0,1001.0,10795897.0,G,"2,450",0,0,0,0,0,0,0,2023-07-28


In [12]:
Data_Denominacion[Data_Denominacion['Grupo art.']=="C338"]

,Grupo art.,Denom.gr.artíc.
488,C338,PT. ARMADO DISNAL AVON FULL


In [13]:
data_Materias

,Grupo art.,Denom.gr.artíc.,Material,Texto breve de material,Gpo.artíc.,Ce.,Alm.,Lote,UMB,Libre utilización,En control calidad,Valor libre util.,Valor en insp.cal.,Bloqueado,Stock no libre,Valor stock bloq.,Valor no libre,Cad./FPC
0,C272,MP.LOCATEL,PB0083218,FRAG. SENSUAL FLOWER AR680583,C272,1000.0,1001.0,10871504.0,KG,2,0,"208,187",0,0,0,0,0,2023-07-06
1,GRIN1,MP Avon,0-02250000,COCOA BUTTER,GRIN1,1000.0,1001.0,10853194.0,G,"18,888",0,0,0,0,0,0,0,2023-07-06
2,C011,MP. Prebel,Y089210000,MARTINIQUE MANEN 92477 - CPL,C011,1000.0,1001.0,10867992.0,KG,20.144,0,"1,563,445",0,0,0,0,0,2023-07-06
3,C011,MP. Prebel,PB0062306,CAPRYLYL CAPRYLATE/CAPRATE,C011,1000.0,1001.0,10878493.0,KG,2.500,0,"273,930",0,0,0,0,0,2023-07-07
4,C274,MP.BEIERSDORF,PB0084145,LICORICE EXTRACT GW/00199,C274,1000.0,1001.0,10839215.0,KG,4.770,0,"1,188,177",0,1,0,"249,094",0,2023-07-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,C041,MP. Natura,PB0091587,50351573 SANTAL ENJOY 295EA,C041,1000.0,1001.0,10882153.0,KG,10.200,0,"4,727,125",0,0,0,0,0,2023-08-02
69,GRIN1,MP Avon,R172059,DIMETHICONE-VINYL DIM.CROSSPOL,GRIN1,1000.0,1001.0,10891761.0,G,42,0,0,0,0,0,0,0,2023-08-04
70,C272,MP.LOCATEL,PB0080854,BASE HIGH TECH FRESHPLEX MEX-EC0380800,C272,1000.0,1001.0,10871663.0,KG,2.875,0,"344,552",0,0,0,0,0,2023-08-04
71,C011,MP. Prebel,PB0045148,MEX-EC0271361 JABON,C011,1000.0,1001.0,10871662.0,KG,4.099,0,"451,942",0,0,0,0,0,2023-08-04
